In [4]:
!pip install fastapi uvicorn


In [1]:
!pip install fastapi uvicorn nest-asyncio -q

import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np
from threading import Thread

nest_asyncio.apply()

app = FastAPI()

# Chargement des modèles
model = joblib.load("modele_prediction_maladie.pkl")
scaler = joblib.load("scaler.pkl")
ohe = joblib.load("onehot_encoder.pkl")
mlb = joblib.load("symptom_encoder.pkl")

# Modèle de données
class Patient(BaseModel):
    age: int
    sexe: str
    fumeur: str
    tension: float
    temperature: float
    symptomes: list[str]

@app.post("/predict")
def predict(patient: Patient):
    if len(patient.symptomes) < 3:
        return {"error": "Au moins 3 symptômes requis."}

    input_num = scaler.transform([[patient.age, patient.tension, patient.temperature]])
    input_cat = ohe.transform([[patient.sexe, patient.fumeur]])
    input_sympt = mlb.transform([patient.symptomes])
    final_input = np.concatenate([input_num, input_cat, input_sympt], axis=1)

    proba = model.predict_proba(final_input)[0]
    classe = model.classes_[np.argmax(proba)]
    return {"maladie": classe, "probabilite": float(np.max(proba))}

# Lancement de FastAPI dans un thread
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

Thread(target=run).start()
